In [3]:
import ee
ee.Authenticate()

Enter verification code:  4/1AdQt8qiGcopTPupkXcKqhZtiFen2z0T454Vs9EKiOI-nH0z1nQ5BtRJkdn8



Successfully saved authorization token.


In [4]:
ee.Initialize()

In [16]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob

# Load data

In [17]:
# read Tree Mosaic Land
tml = ee.ImageCollection('projects/wri-datalab/TML')
tml_tree_cover = tml.reduce(ee.Reducer.mean()).rename('b1')

In [18]:
# get list of urbanshift cities
boundary_georef = pd.read_csv('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv') 

In [19]:
# subset cities with avaialable tml data
tml_available_cities = ['Teresina city','Teresina region','Belem city','Belem region','San Jose','Kigali',
                        'Freetown city','Freetown region','Marrakech','Chennai region','Chennai region',
                        'Pune','Surat','Jakarta','Bitung','Semarang','Balikpapan','Balikpapan']
boundary_georef = boundary_georef.loc[boundary_georef['city_name'].isin(tml_available_cities)].reset_index(drop=True)
boundary_georef                        

,geo_name,level,aoi_boundary_name,units_boundary_name,city_name,country_name,country_code,continent
0,BRA-Teresina,city,ADM4union,ADM4,Teresina city,Brazil,BRA,America
1,BRA-Teresina,region,ADM2union,ADM2,Teresina region,Brazil,BRA,America
2,BRA-Belem,city,ADM4union,ADM4,Belem city,Brazil,BRA,America
3,BRA-Belem,region,ADM2union,ADM2,Belem region,Brazil,BRA,America
4,CRI-San_Jose,region,ADM2union,ADM2,San Jose,Costa Rica,CRI,America
5,RWA-Kigali,city,ADM4union,ADM4,Kigali,Rwanda,RWA,Africa
6,SLE-Freetown,city,ADM4cityunion,ADM4city,Freetown city,Sierra Leone,SLE,Africa
7,SLE-Freetown,region,ADM4regionunion,ADM4region,Freetown region,Sierra Leone,SLE,Africa
8,MAR-Marrakech,NaN,ADM2,ADM5,Marrakech,Morocco,MAR,Africa
9,IND-Chennai,region,ADM4union,ADM4,Chennai region,India,IND,Asia


# Extract TML by city

In [9]:
for i in range(0,len(boundary_georef)):
    print(i)
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    print(boundary_id)
    
    # read boundaries
    boundary_path = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # Download ee.Image for TML as GeoTIFF
    geemap.ee_export_image_to_drive(
        tml_tree_cover, 
        description = boundary_id + '-TML-TreeCover',
        folder='data', 
        scale=10, 
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )

0
BRA-Teresina-ADM4union
1
BRA-Teresina-ADM2union
2
BRA-Belem-ADM4union
3
BRA-Belem-ADM2union
4
CRI-San_Jose-ADM2union
5
RWA-Kigali-ADM4union
6
SLE-Freetown-ADM4cityunion
7
SLE-Freetown-ADM4regionunion
8
MAR-Marrakech-ADM2
9
IND-Chennai-ADM4union
10
IND-Surat-ADM4union
11
IDN-Jakarta-ADM4union
12
IDN-Bitung-ADM2
13
IDN-Semarang-ADM1
14
IDN-Balikpapan-ADM4union


# Upload to aws

Since we can't download directly the rasters locally due to their size, the rasters are stored in a google-drive folder and then downloaded locally in order to push them back to s3 bucket.

In [41]:
# connect to s3
aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

# specify bucket name
bucket_name = 'cities-urbanshift' 

In [10]:
# define work directory
out_dir = os.getcwd()

In [119]:
# upload the geotiff files to aws s3 byucket
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data\\' + boundary_id +'-TML-TreeCover.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/tree_cover/tree_mosaic_land/v_0/'+ boundary_id + '-TML-tree_cover-2000.tif',
                               ExtraArgs={'ACL':'public-read'})


 boundary_id: BRA-Teresina-ADM4union

 boundary_id: BRA-Teresina-ADM2union

 boundary_id: BRA-Belem-ADM4union

 boundary_id: BRA-Belem-ADM2union

 boundary_id: CRI-San_Jose-ADM2union

 boundary_id: RWA-Kigali-ADM4union

 boundary_id: SLE-Freetown-ADM4cityunion

 boundary_id: SLE-Freetown-ADM4regionunion

 boundary_id: MAR-Marrakech-ADM2

 boundary_id: IND-Chennai-ADM4union

 boundary_id: IND-Surat-ADM4union

 boundary_id: IDN-Jakarta-ADM4union

 boundary_id: IDN-Bitung-ADM2

 boundary_id: IDN-Semarang-ADM1

 boundary_id: IDN-Balikpapan-ADM4union


# Visualize data

In [11]:
## create map
Map = geemap.Map(height="350px")
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [12]:
# read boundaries
i = 0
boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
print(boundary_id)
boundary_path = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
boundary_geo = requests.get(boundary_path).json()
boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)

BRA-Teresina-ADM4union


In [13]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(boundary_geo_ee, zoom=8)

In [15]:
Map.addLayer(boundary_geo_ee)

In [14]:
tree_percent_threshold = 10
Map.addLayer(tml_tree_cover.updateMask(tml_tree_cover.gte(tree_percent_threshold)),
             {'min':0, 'max':0.5, 'palette':['white','#006400']},
             'Tree Cover 2020 (WRI Trees in Mosaic Landscapes)',True,1)